# reference data for the "array" dataset

We further consider "FILTER" column in the variant annotation file and clean-up the reference file, `ukb_cal-consequence_wb_maf_gene_ld_indep_mpc_pli.tsv.gz` and save the new file as `ukb_cal-consequence_wb_maf_gene_ld_indep_mpc_pli_qc.tsv.gz`

In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('paramters.sh')

In [3]:
in_f <- '../ukb_cal-consequence_wb_maf_gene_ld_indep_mpc_pli.tsv.gz'
annotation_f <- file.path(data_dir, annotation_array_f)
out_f <- '../ukb_cal-consequence_wb_maf_gene_ld_indep_mpc_pli_qc.tsv.gz'


In [4]:
in_f %>% 
fread(colClasses = c('ld_indep'='character')) %>%
rename_with(function(x){str_replace(x, '#', '')}, starts_with("#")) -> in_df

annotation_f %>% 
fread() %>%
rename_with(function(x){str_replace(x, '#', '')}, starts_with("#")) -> annotation_df


In [5]:
in_df %>%
left_join(
    annotation_df %>%
    mutate(V = paste(CHROM, POS, REF, ALT, sep=':')) %>%
    select(V, FILTER),
    by='V'
) -> merged_df


In [6]:
merged_df %>% count(FILTER)


,FILTER,n
,<chr>,<int>
1,.,651935
2,gnomad_af,164
3,hwe,33792
4,hwe;gnomad_af,3
5,hwe;mgi,3
6,mcpi,10
7,mgi,41
8,missingness,75499
9,missingness;gnomad_af,40


In [7]:
merged_df %>% 
filter(FILTER == '.') %>%
select(-FILTER) %>%
fwrite(str_replace(out_f, '.gz$', ''), sep='\t', na = "NA", quote=F)


In [8]:
sprintf('bgzip -l9 %s', str_replace(out_f, '.gz$', ''))


[1] "bgzip -l9 ../ukb_cal-consequence_wb_maf_gene_ld_indep_mpc_pli_qc.tsv"